In [1]:
import os
import fnmatch
import librosa
import json
import numpy as np


In [2]:
path = './samples'

files = []
for root, dirnames, filenames in os.walk(path):
    for filename in fnmatch.filter(filenames, '*.wav'):
        files.append(os.path.join(root, filename))

print("found %d .wav files in %s"%(len(files),path))


found 979 .wav files in ./samples


In [3]:
files

['./samples/NDS-1_Clap_001.wav',
 './samples/NDS-1_Clap_002.wav',
 './samples/NDS-1_Clap_003.wav',
 './samples/NDS-1_Clap_004.wav',
 './samples/NDS-1_Clap_005.wav',
 './samples/NDS-1_Clap_006.wav',
 './samples/NDS-1_Clap_007.wav',
 './samples/NDS-1_Clap_008.wav',
 './samples/NDS-1_Clap_009.wav',
 './samples/NDS-1_Clap_010.wav',
 './samples/NDS-1_Clap_011.wav',
 './samples/NDS-1_Clap_012.wav',
 './samples/NDS-1_Clap_013.wav',
 './samples/NDS-1_Clap_014.wav',
 './samples/NDS-1_Clap_015.wav',
 './samples/NDS-1_Clap_016.wav',
 './samples/NDS-1_Clap_017.wav',
 './samples/NDS-1_Clap_018.wav',
 './samples/NDS-1_Clap_019.wav',
 './samples/NDS-1_Clap_020.wav',
 './samples/NDS-1_Clap_021.wav',
 './samples/NDS-1_Clap_022.wav',
 './samples/NDS-1_Clap_023.wav',
 './samples/NDS-1_Clap_024.wav',
 './samples/NDS-1_Clap_025.wav',
 './samples/NDS-1_Clap_026.wav',
 './samples/NDS-1_Clap_027.wav',
 './samples/NDS-1_Clap_028.wav',
 './samples/NDS-1_Clap_029.wav',
 './samples/NDS-1_Clap_030.wav',
 './sample

In [9]:
def get_features(y, sr):
    y = y[0:sr]  # analyze just first second
    S = librosa.feature.melspectrogram(y, sr=sr, n_mels=128)
    log_S = librosa.amplitude_to_db(S, ref=np.max)
    mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)
    delta_mfcc = librosa.feature.delta(mfcc, mode='nearest')
    delta2_mfcc = librosa.feature.delta(mfcc, order=2, mode='nearest')
    feature_vector = np.concatenate((np.mean(mfcc,1), np.mean(delta_mfcc,1), np.mean(delta2_mfcc,1)))
    feature_vector = (feature_vector-np.mean(feature_vector)) / np.std(feature_vector)
    return feature_vector

In [10]:
feature_vectors = []
sound_paths = []
for i,f in enumerate(files):
    if i % 100 == 0:
        print("get %d of %d = %s"%(i+1, len(files), f))
    try:
        y, sr = librosa.load(f)
        if len(y) < 2:
            print("error loading %s" % f)
            continue
        feat = get_features(y, sr)
        feature_vectors.append(feat)
        sound_paths.append(f)
    except:
        print("error loading %s" % f)
        
print("calculated %d feature vectors"%len(feature_vectors))


get 1 of 979 = ./samples/NDS-1_Clap_001.wav


/usr/local/lib/python3.7/site-packages/scipy/signal/_savitzky_golay.py:135: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  coeffs, _, _, _ = lstsq(A, y)


get 101 of 979 = ./samples/NDS-1_Clap_101.wav
get 201 of 979 = ./samples/NDS-1_Kick_606_025.wav
get 301 of 979 = ./samples/NDS-1_Kick_808_042.wav
get 401 of 979 = ./samples/NDS-1_Kick_808_142.wav
get 501 of 979 = ./samples/NDS-1_Kick_Drive_019.wav
get 601 of 979 = ./samples/NDS-1_OpenHat_27.wav
get 701 of 979 = ./samples/NDS-1_Perc_Cowbell_59.wav
get 801 of 979 = ./samples/NDS-1_Perc_Syn_167.wav
get 901 of 979 = ./samples/NDS-1_Snare_055.wav
calculated 979 feature vectors


In [15]:
with open("features.json", 'w') as outfile:
    json.dump([[x.tolist() for x in feature_vectors]], outfile)

with open("files.json", 'w') as outfile:
    json.dump([sound_paths], outfile)